# IP모형

## 라이브러리 불러오기

In [1]:
!pip install pulp

In [2]:
!pip install ortoolpy

## 데이터불러오기

In [1]:
import numpy as np
import pandas as pd
from itertools import product
from pulp import LpVariable, lpSum, value
from ortoolpy import model_min, addvars, addvals, model_max

In [2]:
df_train = pd.read_csv('data/train_20221121.csv', sep=',', encoding='cp949')
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121754 entries, 0 to 121753
Data columns (total 10 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   X1      121754 non-null  int64  
 1   X2      121754 non-null  float64
 2   X3      121754 non-null  float64
 3   X4      121754 non-null  float64
 4   X5      121754 non-null  object 
 5   X6      121754 non-null  int64  
 6   X7      121754 non-null  int64  
 7   X8      121754 non-null  object 
 8   Y1      121754 non-null  int64  
 9   Y2      121754 non-null  int64  
dtypes: float64(3), int64(5), object(2)
memory usage: 9.3+ MB


## 데이터 확인

In [3]:
df_data = df_train[['X7', 'Y2']].iloc[:115]

In [4]:
df_pivot = df_data.pivot(columns='X7', values='Y2').transpose().fillna(0)

In [5]:
df_data['Y2'].sum()

230785

In [6]:
ct = tuple(df_data['Y2'])
ct

(2241,
 1163,
 1094,
 2604,
 1564,
 3012,
 1202,
 1161,
 791,
 5163,
 594,
 1150,
 3413,
 2361,
 4329,
 565,
 1242,
 3887,
 856,
 2496,
 4868,
 1950,
 551,
 3209,
 1960,
 4563,
 1959,
 511,
 2226,
 7443,
 544,
 1222,
 1704,
 1902,
 1041,
 2745,
 5842,
 2120,
 1217,
 2057,
 1733,
 3687,
 1306,
 1770,
 2492,
 1279,
 1135,
 817,
 999,
 1653,
 1086,
 1215,
 1774,
 3436,
 2611,
 1098,
 702,
 1027,
 2774,
 2990,
 1109,
 583,
 835,
 2166,
 1111,
 1553,
 1275,
 1179,
 3325,
 690,
 1617,
 1766,
 862,
 1194,
 5147,
 862,
 1172,
 1002,
 819,
 1882,
 4687,
 556,
 1623,
 92,
 2999,
 1360,
 2669,
 1265,
 920,
 927,
 2753,
 748,
 2170,
 1281,
 4824,
 5303,
 2883,
 1350,
 1386,
 950,
 914,
 1431,
 1001,
 3355,
 2035,
 5460,
 677,
 954,
 3688,
 1136,
 868,
 8061,
 1150,
 1978,
 1076)

In [7]:
df_pivot.shape

(12, 115)

In [8]:
df_pivot.astype('int')

,0,1,2,3,4,5,6,7,8,9,...,105,106,107,108,109,110,111,112,113,114
X7,,,,,,,,,,,,,,,,,,,,,
14,0,0,1094,0,0,0,0,0,0,0,...,0,0,954,0,0,0,0,0,0,0
21,0,0,0,0,0,0,0,0,0,5163,...,0,0,0,0,0,868,0,0,0,0
22,0,0,0,0,0,0,0,0,0,0,...,0,677,0,0,0,0,0,1150,0,0
23,0,0,0,0,1564,0,0,0,0,0,...,5460,0,0,0,0,0,8061,0,0,0
24,0,0,0,2604,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1076
26,0,0,0,0,0,3012,0,0,791,0,...,0,0,0,0,0,0,0,0,0,0
27,0,1163,0,0,0,0,0,1161,0,0,...,0,0,0,0,0,0,0,0,0,0
91,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1136,0,0,0,0,0


In [9]:
rows = len(df_pivot.index)
columns = len(df_pivot.columns)
pr = list(product(range(rows), range(columns)))
ct_mean_workspace = sum(ct) / rows

In [10]:
cond = 100

## IP모형 설계

In [13]:
# 목적함수 정의
m1 = model_max()
v1 = {(i, j) : LpVariable('v%d_%d'%(i, j), lowBound=0, upBound=1, cat='Integer') for i, j in pr}
m1 += lpSum(ct[i] * v1[0,j] for i, j in pr)

# 제약조건 정의
for i in range(rows):
    m1 += lpSum(ct[j] * v1[i,j] for j in range(columns)) <= ct_mean_workspace + cond
    m1 += lpSum(ct[j] * v1[i,j] for j in range(columns)) >= ct_mean_workspace - cond
for j in range(columns):
    m1 += lpSum(v1[i,j] for i in range(rows)) == 1

In [14]:
m1

NoName:
MAXIMIZE
21739*v0_0 + 21739*v0_1 + 21739*v0_10 + 21739*v0_11 + 21739*v0_12 + 21739*v0_13 + 21739*v0_14 + 21739*v0_15 + 21739*v0_16 + 21739*v0_17 + 21739*v0_18 + 21739*v0_19 + 21739*v0_2 + 21739*v0_20 + 21739*v0_21 + 21739*v0_22 + 21739*v0_23 + 21739*v0_24 + 21739*v0_25 + 21739*v0_26 + 21739*v0_27 + 21739*v0_28 + 21739*v0_29 + 21739*v0_3 + 21739*v0_30 + 21739*v0_31 + 21739*v0_32 + 21739*v0_33 + 21739*v0_34 + 21739*v0_35 + 21739*v0_36 + 21739*v0_37 + 21739*v0_38 + 21739*v0_39 + 21739*v0_4 + 21739*v0_40 + 21739*v0_41 + 21739*v0_42 + 21739*v0_43 + 21739*v0_44 + 21739*v0_45 + 21739*v0_46 + 21739*v0_47 + 21739*v0_48 + 21739*v0_49 + 21739*v0_5 + 21739*v0_50 + 21739*v0_51 + 21739*v0_52 + 21739*v0_53 + 21739*v0_54 + 21739*v0_55 + 21739*v0_56 + 21739*v0_57 + 21739*v0_58 + 21739*v0_59 + 21739*v0_6 + 21739*v0_60 + 21739*v0_61 + 21739*v0_62 + 21739*v0_63 + 21739*v0_64 + 21739*v0_65 + 21739*v0_66 + 21739*v0_67 + 21739*v0_68 + 21739*v0_69 + 21739*v0_7 + 21739*v0_70 + 21739*v0_71 + 21739*v0_72

In [15]:
m1.solve()

1

In [16]:
df_tr_sol = df_pivot.copy()

In [17]:
for k,x in v1.items():
    i,j = k[0], k[1]
    df_tr_sol.iloc[i][j] = value(x)
    
display(df_tr_sol)

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
X7,,,,,,,,,,,,,,,,,,,,,
14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
21,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
91,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
df_sum = df_tr_sol * ct

In [19]:
df_sum

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
X7,,,,,,,,,,,,,,,,,,,,,
14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,791.0,0.0,...,0.0,748.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,950.0
21,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22,0.0,0.0,0.0,0.0,0.0,3012.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23,0.0,0.0,0.0,0.0,1564.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4824.0,0.0,0.0,0.0,0.0,0.0
24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,5303.0,0.0,0.0,0.0,0.0
25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5163.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2170.0,0.0,0.0,0.0,0.0,1350.0,0.0,0.0
91,0.0,0.0,1094.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1281.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
max_worktime = 0
for i in range(rows):
    k = sum(df_sum.iloc[i])
    print(k)
    if k > max_worktime:
        max_worktime = k
        
print("Makespan: ",max_worktime)
    

16403.0
16370.0
16348.0
16338.0
16464.0
16500.0
16323.0
16377.0
16501.0
16446.0
16435.0
16496.0
Makespan:  16501.0


In [21]:
k = 0
for i in range(rows):
    k += sum(df_sum.iloc[i])
k

197001.0

## IP모형 생성 함수

In [11]:
df_tr_sol = df_pivot.copy()

def lpModel(df_data, cond):
    df_pivot = df_data.pivot(columns='X7', values='Y2').transpose().fillna(0)
    ct = tuple(df_data['Y2'])
    df_pivot.astype('int')
    rows = len(df_pivot.index)
    columns = len(df_pivot.columns)
    pr = list(product(range(rows), range(columns)))
    ct_mean_workspace = sum(ct) / rows
    
    m1 = model_max()
    v1 = {(i, j) : LpVariable('v%d_%d'%(i, j), lowBound=0, upBound=1, cat='Integer') for i, j in pr}
    m1 += lpSum(ct[i] * v1[0,j] for i, j in pr)
    for i in range(rows):
        m1 += lpSum(ct[j] * v1[i,j] for j in range(columns)) <= ct_mean_workspace + cond
        m1 += lpSum(ct[j] * v1[i,j] for j in range(columns)) >= ct_mean_workspace - cond
    for j in range(columns):
        m1 += lpSum(v1[i,j] for i in range(rows)) == 1
    
    m1.solve()
    df_tr_sol = df_pivot.copy()
    for k,x in v1.items():
        i,j = k[0], k[1]
        df_tr_sol.iloc[i][j] = value(x)
    df_sum = df_tr_sol * ct
    
    return df_sum

## 총완료시간 계산함수

In [12]:
def cal_makespan(df_sum):
    max_worktime = 0
    for i in range(rows):
        k = sum(df_sum.iloc[i])
        if k > max_worktime:
            max_worktime = k
    return max_worktime
    

## 휴리스틱 기법 활용

휴리스틱기법으로 상한, 하한을 찾고 최적의 할당방법을 찾는다.

In [13]:
condlst = [i for i in range(100, 301, 50)]
best_makespan = 100000

for cond in condlst:
    df_sol = lpModel(df_data, cond)
    makespan = cal_makespan(df_sol)
    if best_makespan > makespan:
        best_makespan = makespan
        best_sol = df_sol
        
print(best_makespan)
best_sol

19326.0


,0,1,2,3,4,5,6,7,8,9,...,105,106,107,108,109,110,111,112,113,114
X7,,,,,,,,,,,,,,,,,,,,,
14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,791.0,0.0,...,0.0,677.0,954.0,0.0,0.0,868.0,0.0,0.0,0.0,0.0
21,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5460.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1076.0
22,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25,2241.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26,0.0,1163.0,0.0,2604.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
91,0.0,0.0,0.0,0.0,1564.0,3012.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3688.0,1136.0,0.0,0.0,0.0,0.0,0.0
